In [1]:
import numpy as np
import pandas as pd
from load_data import load_train_data
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from gpu import set_gpu

set_gpu()

Using TensorFlow backend.
Num GPUs Available:  1


In [2]:
# Load data
feat,params,daq = load_train_data('train_data_raw_AB.mat')

## Run ANN

In [3]:
# Build NN model
model = Sequential([
    Dense(units=60, input_shape=(60,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=7, activation='softmax')
])
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 60)                3660      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1952      
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 231       
Total params: 5,843
Trainable params: 5,843
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Index data
# grp = 1 (static), 2 (dynamic), 3 (0g), 4 (400g), 5 (500g), 6 (600g)
sub = 14
grp = 1
ind = (params[:,0] == sub) & (params[:,3] == grp)

# Shuffle and split data
feat_s, label_s = shuffle(feat[ind,:],params[ind,-2,np.newaxis])
X_train, X_test, y_train, y_test = train_test_split(feat_s, label_s, test_size=0.3, random_state=0, stratify=label_s)
X_train, X_test = feat_s, feat_s
y_train, y_test = label_s, label_s

# Reshape and scale data
y_train = np.squeeze(y_train-1)
y_test = np.squeeze(y_test-1)
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(X_train)
X_train_n = scaler.transform(X_train)
X_test_n = scaler.transform(X_test)

In [7]:
# Fit model
model.fit(x=X_train_n, y=y_train, batch_size=32, epochs=30, verbose=2)

Epoch 1/30
110/110 - 0s - loss: 0.0898 - accuracy: 0.9846
Epoch 2/30
110/110 - 0s - loss: 0.0848 - accuracy: 0.9860
Epoch 3/30
110/110 - 0s - loss: 0.0802 - accuracy: 0.9857
Epoch 4/30
110/110 - 0s - loss: 0.0760 - accuracy: 0.9863
Epoch 5/30
110/110 - 0s - loss: 0.0721 - accuracy: 0.9863
Epoch 6/30
110/110 - 0s - loss: 0.0686 - accuracy: 0.9866
Epoch 7/30
110/110 - 0s - loss: 0.0656 - accuracy: 0.9874
Epoch 8/30
110/110 - 0s - loss: 0.0624 - accuracy: 0.9877
Epoch 9/30
110/110 - 0s - loss: 0.0597 - accuracy: 0.9886
Epoch 10/30
110/110 - 0s - loss: 0.0570 - accuracy: 0.9886
Epoch 11/30
110/110 - 0s - loss: 0.0545 - accuracy: 0.9897
Epoch 12/30
110/110 - 0s - loss: 0.0520 - accuracy: 0.9906
Epoch 13/30
110/110 - 0s - loss: 0.0499 - accuracy: 0.9903
Epoch 14/30
110/110 - 0s - loss: 0.0477 - accuracy: 0.9914
Epoch 15/30
110/110 - 0s - loss: 0.0461 - accuracy: 0.9917
Epoch 16/30
110/110 - 0s - loss: 0.0441 - accuracy: 0.9920
Epoch 17/30
110/110 - 0s - loss: 0.0423 - accuracy: 0.9926
Epoch 

In [8]:
# Evaluate test set
out = np.argmax(model.predict(x=X_test_n), axis = 1)
np.sum(out.reshape(y_test.shape) == y_test)/y_test.shape[0]

0.996

## Run LDA

In [3]:
from lda import train_lda, predict

In [4]:
sub = 14
grp = 1
ind = (params[:,0] == sub) & (params[:,3] == grp)
X_train, y_train = shuffle(feat[ind,:],params[ind,-2])
# X_train = feat[ind,:]
# y_train = params[ind,-2] - 1
y_train_lda = y_train[...,np.newaxis] - 1
y_test_lda = y_train[...,np.newaxis] - 1

In [5]:
w,c = train_lda(X_train,y_train_lda)
# X_test_n = scaler.transform(X_test)
out = predict(X_train,w,c)
np.sum(out.reshape(y_test_lda.shape) == y_test_lda)/y_test_lda.shape[0]

0.9928571428571429

## Run CNN

In [3]:
from load_data import process_daq


In [4]:
raw = process_daq(daq,params)
raw = raw[::2,:,:].transpose(2,1,0)
sub = 14
grp = 1
ind = (params[:,0] == sub) & (params[:,3] == grp)

In [5]:
feat_s, label_s = shuffle(raw[ind,:,:],params[ind,-2])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(feat_s, label_s, test_size=0.3, random_state=0, stratify=label_s)


In [6]:
X_train, X_test = feat_s, feat_s
y_train, y_test = label_s, label_s
scaler = MinMaxScaler(feature_range=(-1,1))
X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0],-1)).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(X_test.shape[0],-1)).reshape(X_test.shape)
# X_train = X_train.transpose(1,2,0)
# X_test = X_test.transpose(1,2,0)
X_train = X_train[...,np.newaxis]
X_test = X_test[...,np.newaxis]
y_train = np.squeeze(y_train-1)

In [7]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), padding='same', input_shape=(6,100,1)),
    BatchNormalization(),
    Activation('relu'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Conv2D(filters=32, kernel_size=(3,3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Flatten(),
    Dense(units=7, activation='softmax'),
])


In [8]:
model.summary()
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 6, 100, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 6, 100, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 6, 100, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 50, 32)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 50, 32)         9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 3, 50, 32)         128       
_________________________________________________________________
activation_1 (Activation)    (None, 3, 50, 32)         0

In [10]:
model.fit(x=X_train, y=y_train, batch_size=32, epochs=30, verbose=2)

Epoch 1/30
110/110 - 0s - loss: 0.4290 - accuracy: 0.8923
Epoch 2/30
110/110 - 0s - loss: 0.3940 - accuracy: 0.9043
Epoch 3/30
110/110 - 0s - loss: 0.3579 - accuracy: 0.9163
Epoch 4/30
110/110 - 0s - loss: 0.3254 - accuracy: 0.9191
Epoch 5/30
110/110 - 0s - loss: 0.2941 - accuracy: 0.9357
Epoch 6/30
110/110 - 0s - loss: 0.2720 - accuracy: 0.9426
Epoch 7/30
110/110 - 0s - loss: 0.2535 - accuracy: 0.9457
Epoch 8/30
110/110 - 0s - loss: 0.2353 - accuracy: 0.9491
Epoch 9/30
110/110 - 0s - loss: 0.2203 - accuracy: 0.9540
Epoch 10/30
110/110 - 0s - loss: 0.2070 - accuracy: 0.9571
Epoch 11/30
110/110 - 0s - loss: 0.1950 - accuracy: 0.9591
Epoch 12/30
110/110 - 0s - loss: 0.1782 - accuracy: 0.9634
Epoch 13/30
110/110 - 0s - loss: 0.1693 - accuracy: 0.9680
Epoch 14/30
110/110 - 0s - loss: 0.1625 - accuracy: 0.9674
Epoch 15/30
110/110 - 0s - loss: 0.1516 - accuracy: 0.9720
Epoch 16/30
110/110 - 0s - loss: 0.1413 - accuracy: 0.9757
Epoch 17/30
110/110 - 0s - loss: 0.1370 - accuracy: 0.9740
Epoch 

In [11]:
out = np.argmax(model.predict(x=X_train), axis = 1)
np.sum(out.reshape(y_train.shape) == y_train)/y_train.shape[0]

0.9937142857142857